In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPool2D(strides=2))
model.add(tf.keras.layers.Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(84, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.build()
model.summary()

In [ ]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
rnd_index = np.arange(train_labels.shape[0])
np.random.shuffle(rnd_index)
n_datasets = 120
ni = int(np.shape(train_labels)[0]/n_datasets)
train_images = train_images[rnd_index,:,:]
train_labels = train_labels[rnd_index]
train_images = np.reshape(train_images,(n_datasets,ni,28,28))
train_labels = np.reshape(train_labels,(n_datasets,ni))
print(ni)

In [ ]:
def reinitialize(model):
    for l in model.layers:
        if hasattr(l,"kernel_initializer"):
            l.kernel.assign(l.kernel_initializer(tf.shape(l.kernel)))
        if hasattr(l,"bias_initializer"):
            l.bias.assign(l.bias_initializer(tf.shape(l.bias)))
        if hasattr(l,"recurrent_initializer"):
            l.recurrent_kernel.assign(l.recurrent_initializer(tf.shape(l.recurrent_kernel)))

In [ ]:
epochs = 100
CEs = np.zeros(n_datasets)
CEp = np.zeros(n_datasets)
Rp_s = np.zeros(n_datasets)
Rp_h = np.zeros(n_datasets)
Rs_s = np.zeros(n_datasets)
Rs_h = np.zeros(n_datasets)
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4))
for j in range(n_datasets):
  reinitialize(model)
  h = model.fit(x=train_images[j,:,:,:], y=train_labels[j,:], batch_size = 64, epochs=epochs)

  CEs[j],A = model.evaluate(x=train_images[j,:,:,:], y=train_labels[j,:])
  Rs_h[j] = 1.0-A
  predicciones = model.predict(train_images[j,:,:,:])
  lista = [predicciones[k,train_labels[j,k]] for k in range(np.shape(predicciones)[0])]
  Rs_s[j] = 1.0-np.mean(np.array(lista))

  CEp[j],A = model.evaluate(x=test_images, y=test_labels)
  Rp_h[j] = 1.0-A
  predicciones = model.predict(test_images)
  lista = [predicciones[k,test_labels[k]] for k in range(np.shape(predicciones)[0])]
  Rp_s[j] = 1.0-np.mean(np.array(lista))

#from google.colab import drive
#drive.mount('/content/drive')
#np.savez('/content/drive/My Drive/rta2712.npz', CEs = CEs, CEp = CEp, Rp_s = Rp_s, Rp_h = Rp_h, Rs_s = Rs_s, Rs_h = Rs_h)


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#npzfile = np.load('/content/drive/My Drive/rta2712.npz')
#CEs = npzfile["CEs"]
#CEp = npzfile["CEp"]
#Rp_s = npzfile["Rp_s"]
#Rp_h = npzfile["Rp_h"]
#Rs_s = npzfile["Rs_s"]
#Rs_h = npzfile["Rs_h"]

CEtilde = -np.log(1.0-Rp_s)
gen = CEtilde-CEs
n_datasets = np.shape(CEs)[0]

In [ ]:
pts=10000
tmin=1e-5
tmax=1000
nmax = 18 # de deltas
Cn = np.zeros(pts)
cota = np.zeros((pts,nmax))
cuantil = np.zeros(nmax)
cuantil01 = np.zeros(nmax)
for idx, t in enumerate(np.linspace(tmin,tmax,pts)):
  Cn[idx]=np.mean(np.exp(t*(CEtilde-CEs)))
  for nd in range(1,nmax+1):
    delta = nd / n_datasets
    cuantil[nd-1] = np.sort(gen)[n_datasets-nd-1]
    cota[idx,nd-1] = np.log(Cn[idx]/delta) / t
    cuantil01[nd-1] = np.sort(Rp_h-Rs_h)[n_datasets-nd-1]


In [ ]:
minimo = np.min(cota[:,5])
donde = np.argmin(cota[:,5])

ejex = np.linspace(tmin,tmax,pts)
fig, ax = plt.subplots()
ax.plot(ejex,cota[:,5],label = r'$\frac{1}{t}\;\log\;\frac{C_n(t)}{\delta}$') #delta=0.05
ax.plot([tmin,tmax],[cuantil[5],cuantil[5]], label = r'$(1-\delta)$-quantile of CE gap',linestyle='dashed')
ax.plot([tmin,tmax],[cuantil01[5],cuantil01[5]], label = r'$(1-\delta)$-quantile of $R_p^\ast$ gap',linestyle='dotted')
plt.plot(ejex[donde], minimo, marker="o", markersize=5, markeredgecolor="blue", markerfacecolor="blue",label = "Minimum value")
ax.legend()
plt.ylim((0,0.8))
ax.set_xlabel("t")
ax.set_ylabel("Generalization gap")

#plt.savefig("cnvst.pdf")
#from google.colab import files
#files.download('cnvst.pdf')